# K₇ Laplacian Comparison: Δ₀ (Scalar) vs Δ₁ (1-Forms)

**Objective**: Test whether the Hodge Laplacian on 1-forms gives λ₁ × H* closer to 13 or 14 than the scalar Laplacian.

**GIFT Prediction**: λ₁ × H* = dim(G₂) - 1 = 13 (or possibly 14 = dim(G₂))

**Hardware**: A100 GPU recommended for N > 5000 samples

---

In [ ]:
# @title 1. Setup & Dependencies
!pip install -q scipy numpy matplotlib scikit-learn

import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, diags
from scipy.sparse.linalg import eigsh
from scipy.spatial import KDTree
from sklearn.neighbors import kneighbors_graph
import time
import json
from dataclasses import dataclass
from typing import Tuple, List, Optional

# Check GPU
import subprocess
try:
    gpu_info = subprocess.check_output(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader']).decode()
    print(f"GPU detected: {gpu_info.strip()}")
except:
    print("No GPU detected - running on CPU (slower)")

print("Setup complete!")

In [ ]:
# @title 2. GIFT Constants

# Topological invariants
DIM_G2 = 14          # dim(G₂) holonomy group
DIM_K7 = 7           # dim(K₇) manifold
B2 = 21              # Second Betti number of K₇
B3 = 77              # Third Betti number of K₇
H_STAR = B2 + B3 + 1 # = 99, cohomological dimension

# Metric properties
DET_G = 65/32        # G₂ metric determinant (exact)

# Canonical ratio from TCS construction
RATIO_CANONICAL = H_STAR / (6 * DIM_G2)  # = 99/84 = 33/28 ≈ 1.179

# Target predictions
TARGET_13 = 13       # dim(G₂) - 1
TARGET_14 = DIM_G2   # dim(G₂)

print(f"GIFT Constants:")
print(f"  dim(G₂) = {DIM_G2}")
print(f"  dim(K₇) = {DIM_K7}")
print(f"  b₂ = {B2}, b₃ = {B3}")
print(f"  H* = b₂ + b₃ + 1 = {H_STAR}")
print(f"  det(g) = {DET_G} = {65}/{32}")
print(f"  Canonical ratio = {RATIO_CANONICAL:.4f}")
print(f"\nTargets: λ₁ × H* = {TARGET_13} or {TARGET_14}")

In [ ]:
# @title 3. K₇ Manifold Sampling (TCS Construction)

def sample_S3(n: int, rng: np.random.Generator) -> np.ndarray:
    """Sample uniformly from 3-sphere."""
    x = rng.standard_normal((n, 4))
    return x / np.linalg.norm(x, axis=1, keepdims=True)

def sample_T4(n: int, rng: np.random.Generator) -> np.ndarray:
    """Sample uniformly from 4-torus."""
    return 2 * np.pi * rng.random((n, 4))

def sample_K7_TCS(n: int, ratio: float, seed: int = 42) -> np.ndarray:
    """
    Sample from K₇ using Twisted Connected Sum (TCS) construction.
    
    K₇ = (S³ × T⁴) ∪_φ (S³ × T⁴) glued along S³ × T³
    
    Args:
        n: Number of samples
        ratio: Controls the relative size of S³ vs T⁴ components
        seed: Random seed
    
    Returns:
        Array of shape (n, 7) - points on K₇
    """
    rng = np.random.default_rng(seed)
    
    # Split samples between two halves
    n1 = n // 2
    n2 = n - n1
    
    points = []
    
    for n_half, sign in [(n1, 1), (n2, -1)]:
        # Sample S³ component (rescaled by ratio)
        s3 = sample_S3(n_half, rng) * ratio
        
        # Sample T⁴ component (take 3 coordinates for 7D total)
        t4 = sample_T4(n_half, rng)[:, :3]
        
        # Combine: use first 3 coords of S³ + T³ + 1 coord from S³
        # This gives 3 + 3 + 1 = 7 dimensions
        half_points = np.column_stack([
            s3[:, :3],           # First 3 from S³
            t4,                   # 3 from T⁴ (reduced to T³)
            sign * s3[:, 3:4]    # Last 1 from S³ (with twist)
        ])
        points.append(half_points)
    
    return np.vstack(points)

def apply_G2_metric(points: np.ndarray) -> np.ndarray:
    """
    Apply G₂-invariant metric deformation.
    
    The metric has det(g) = 65/32 (GIFT prediction).
    We apply anisotropic scaling to achieve this determinant.
    """
    # Scale factors to achieve det(g) = 65/32
    # det = product of scales^2 in each direction
    target_det = DET_G
    
    # Use G₂-motivated scaling: (1, 1, 1, α, α, α, β) with constraints
    # α³ × β = target_det^(1/2) for unit base
    alpha = (target_det ** (1/7)) ** 0.5
    beta = target_det ** (1/14)
    
    scales = np.array([1, 1, 1, alpha, alpha, alpha, beta])
    
    return points * scales

# Test sampling
test_points = sample_K7_TCS(1000, RATIO_CANONICAL)
print(f"K₇ sampling test: {test_points.shape}")
print(f"  Mean: {test_points.mean(axis=0)}")
print(f"  Std:  {test_points.std(axis=0)}")

In [ ]:
# @title 4. Reference Manifold Samplers (for Calibration)

def sample_sphere(n: int, dim: int, seed: int = 42) -> np.ndarray:
    """Sample uniformly from S^dim (embedded in R^{dim+1})."""
    rng = np.random.default_rng(seed)
    x = rng.standard_normal((n, dim + 1))
    return x / np.linalg.norm(x, axis=1, keepdims=True)

def sample_torus(n: int, dim: int, seed: int = 42) -> np.ndarray:
    """Sample uniformly from T^dim (flat torus)."""
    rng = np.random.default_rng(seed)
    return 2 * np.pi * rng.random((n, dim))

# Known eigenvalues for calibration
REFERENCE_EIGENVALUES = {
    'S3': {'lambda1': 3.0, 'dim': 4},    # First nonzero eigenvalue of S³
    'S7': {'lambda1': 7.0, 'dim': 8},    # First nonzero eigenvalue of S⁷
    'T7': {'lambda1': 1.0, 'dim': 7},    # First nonzero eigenvalue of T⁷ (with 2π period)
}

print("Reference manifolds for calibration:")
for name, info in REFERENCE_EIGENVALUES.items():
    print(f"  {name}: λ₁ = {info['lambda1']}, embedding dim = {info['dim']}")

In [ ]:
# @title 5. Graph Laplacian Construction

def build_graph_laplacian(
    points: np.ndarray,
    k: int = 20,
    epsilon: Optional[float] = None
) -> csr_matrix:
    """
    Build the graph Laplacian from point cloud.
    
    Uses k-NN graph with Gaussian weights.
    
    Args:
        points: (N, d) array of points
        k: Number of nearest neighbors
        epsilon: Bandwidth for Gaussian kernel (auto if None)
    
    Returns:
        Sparse Laplacian matrix L = D - W
    """
    n = len(points)
    
    # Build k-NN graph
    tree = KDTree(points)
    distances, indices = tree.query(points, k=k+1)  # +1 to exclude self
    
    # Auto-select epsilon based on median distance
    if epsilon is None:
        epsilon = np.median(distances[:, 1:]) ** 2
    
    # Build weight matrix with Gaussian kernel
    rows, cols, data = [], [], []
    
    for i in range(n):
        for j_idx in range(1, k+1):  # Skip self (index 0)
            j = indices[i, j_idx]
            d = distances[i, j_idx]
            w = np.exp(-d**2 / epsilon)
            
            rows.append(i)
            cols.append(j)
            data.append(w)
    
    W = csr_matrix((data, (rows, cols)), shape=(n, n))
    W = (W + W.T) / 2  # Symmetrize
    
    # Degree matrix
    D = diags(np.array(W.sum(axis=1)).flatten())
    
    # Laplacian L = D - W
    L = D - W
    
    return L, epsilon

def compute_scalar_eigenvalues(
    L: csr_matrix,
    n_eigenvalues: int = 10
) -> np.ndarray:
    """
    Compute smallest eigenvalues of scalar Laplacian Δ₀.
    
    Returns:
        Array of eigenvalues (sorted, smallest first)
    """
    eigenvalues, _ = eigsh(L, k=n_eigenvalues, which='SM')
    return np.sort(eigenvalues)

print("Graph Laplacian functions defined.")

In [ ]:
# @title 6. Hodge Laplacian on 1-Forms (Δ₁)

def build_exterior_derivative(
    points: np.ndarray,
    k: int = 20,
    epsilon: Optional[float] = None
) -> Tuple[csr_matrix, float]:
    """
    Build discrete exterior derivative d₀: Ω⁰ → Ω¹.
    
    For a graph, d₀ is the incidence matrix:
    (d₀f)(e_{ij}) = f(j) - f(i) for edge e_{ij}
    
    Returns:
        d0: Sparse matrix (n_edges, n_vertices)
        epsilon: Bandwidth used
    """
    n = len(points)
    
    # Build k-NN graph
    tree = KDTree(points)
    distances, indices = tree.query(points, k=k+1)
    
    if epsilon is None:
        epsilon = np.median(distances[:, 1:]) ** 2
    
    # Collect edges (directed, i < j to avoid duplicates)
    edges = set()
    edge_weights = {}
    
    for i in range(n):
        for j_idx in range(1, k+1):
            j = indices[i, j_idx]
            d = distances[i, j_idx]
            w = np.exp(-d**2 / (2 * epsilon))
            
            edge = (min(i, j), max(i, j))
            if edge not in edges:
                edges.add(edge)
                edge_weights[edge] = w
            else:
                # Average weights for bidirectional edges
                edge_weights[edge] = (edge_weights[edge] + w) / 2
    
    edges = sorted(edges)
    n_edges = len(edges)
    
    # Build incidence matrix d₀
    rows, cols, data = [], [], []
    
    for e_idx, (i, j) in enumerate(edges):
        w = np.sqrt(edge_weights[(i, j)])
        rows.extend([e_idx, e_idx])
        cols.extend([i, j])
        data.extend([-w, w])  # d₀f(e) = w*(f(j) - f(i))
    
    d0 = csr_matrix((data, (rows, cols)), shape=(n_edges, n))
    
    return d0, epsilon, edges, edge_weights

def build_hodge_laplacian_1form(
    points: np.ndarray,
    k: int = 20,
    epsilon: Optional[float] = None
) -> Tuple[csr_matrix, float]:
    """
    Build Hodge Laplacian on 1-forms: Δ₁ = d₀ᵀd₀ + d₁d₁ᵀ
    
    For simplicity (no 2-forms), we use: Δ₁ ≈ d₀ᵀd₀
    This captures the "curl-free" part of 1-forms.
    
    Returns:
        L1: Hodge Laplacian on 1-forms (n_edges × n_edges)
        epsilon: Bandwidth used
    """
    d0, epsilon, edges, edge_weights = build_exterior_derivative(points, k, epsilon)
    
    # Δ₁ = d₀ᵀ d₀ (this is the "down" Laplacian on 1-forms)
    # Note: Full Hodge Laplacian would include d₁d₁ᵀ term
    L1_down = d0 @ d0.T
    
    # For a more complete approximation, we also add the "up" Laplacian
    # This requires building d₁, which needs triangles
    # For now, we use just the down part (sufficient for gauge theory)
    
    return L1_down, epsilon, len(edges)

def compute_1form_eigenvalues(
    L1: csr_matrix,
    n_eigenvalues: int = 10
) -> np.ndarray:
    """
    Compute smallest eigenvalues of 1-form Laplacian Δ₁.
    """
    # L1 may have many zero eigenvalues (harmonic forms)
    # We compute a few more to find the first nonzero
    n_compute = min(n_eigenvalues + 50, L1.shape[0] - 1)
    eigenvalues, _ = eigsh(L1, k=n_compute, which='SM')
    return np.sort(eigenvalues)

print("Hodge Laplacian functions defined.")

In [ ]:
# @title 7. Calibration Protocol (ε-rescaling)

@dataclass
class CalibrationResult:
    manifold: str
    true_lambda1: float
    raw_lambda1: float
    scale_factor: float
    calibrated_lambda1: float
    error_percent: float

def calibrate_on_reference(
    manifold: str,
    n: int = 2000,
    k: int = 20,
    seed: int = 42
) -> CalibrationResult:
    """
    Calibrate the graph Laplacian on a reference manifold.
    
    Returns scale factor to convert raw eigenvalues to true eigenvalues.
    """
    info = REFERENCE_EIGENVALUES[manifold]
    true_lambda1 = info['lambda1']
    dim = info['dim']
    
    # Sample manifold
    if manifold.startswith('S'):
        sphere_dim = int(manifold[1:])
        points = sample_sphere(n, sphere_dim, seed)
    elif manifold.startswith('T'):
        torus_dim = int(manifold[1:])
        points = sample_torus(n, torus_dim, seed)
    else:
        raise ValueError(f"Unknown manifold: {manifold}")
    
    # Build Laplacian and compute eigenvalues
    L, epsilon = build_graph_laplacian(points, k=k)
    eigenvalues = compute_scalar_eigenvalues(L, n_eigenvalues=5)
    
    # First nonzero eigenvalue (skip the zero eigenvalue)
    raw_lambda1 = eigenvalues[1] if eigenvalues[0] < 1e-10 else eigenvalues[0]
    
    # Scale factor: true / raw
    scale_factor = true_lambda1 / raw_lambda1
    calibrated = raw_lambda1 * scale_factor
    error = abs(calibrated - true_lambda1) / true_lambda1 * 100
    
    return CalibrationResult(
        manifold=manifold,
        true_lambda1=true_lambda1,
        raw_lambda1=raw_lambda1,
        scale_factor=scale_factor,
        calibrated_lambda1=calibrated,
        error_percent=error
    )

def run_calibration(n: int = 2000, k: int = 20) -> dict:
    """Run calibration on all reference manifolds."""
    print("="*60)
    print("CALIBRATION ON REFERENCE MANIFOLDS")
    print("="*60)
    
    results = {}
    scale_factors = []
    
    for manifold in ['S3', 'S7', 'T7']:
        result = calibrate_on_reference(manifold, n=n, k=k)
        results[manifold] = result
        scale_factors.append(result.scale_factor)
        
        print(f"\n{manifold}:")
        print(f"  True λ₁ = {result.true_lambda1}")
        print(f"  Raw λ₁  = {result.raw_lambda1:.6f}")
        print(f"  Scale   = {result.scale_factor:.4f}")
    
    # Average scale factor
    avg_scale = np.mean(scale_factors)
    std_scale = np.std(scale_factors)
    
    print(f"\n" + "-"*60)
    print(f"Average scale factor: {avg_scale:.4f} ± {std_scale:.4f}")
    print("="*60)
    
    return results, avg_scale

# Run calibration
calibration_results, SCALE_FACTOR = run_calibration(n=2000, k=20)

In [ ]:
# @title 8. Main Experiment: Δ₀ vs Δ₁ on K₇

@dataclass
class K7Result:
    N: int
    k: int
    ratio: float
    
    # Scalar Laplacian Δ₀
    lambda1_0_raw: float
    lambda1_0_scaled: float
    product_0: float  # λ₁ × H*
    deviation_0_from_13: float
    deviation_0_from_14: float
    
    # 1-Form Laplacian Δ₁
    lambda1_1_raw: float
    lambda1_1_scaled: float
    product_1: float  # λ₁ × H*
    deviation_1_from_13: float
    deviation_1_from_14: float
    
    # Metadata
    n_edges: int
    time_0: float
    time_1: float

def run_K7_experiment(
    N: int = 5000,
    k: int = 30,
    ratio: float = RATIO_CANONICAL,
    scale_factor: float = 1.0,
    seed: int = 42,
    apply_metric: bool = True
) -> K7Result:
    """
    Run the main experiment comparing Δ₀ and Δ₁ on K₇.
    """
    print(f"\nSampling K₇ with N={N}, ratio={ratio:.4f}...")
    
    # Sample K₇
    points = sample_K7_TCS(N, ratio, seed)
    if apply_metric:
        points = apply_G2_metric(points)
    
    print(f"  Points shape: {points.shape}")
    
    # === Scalar Laplacian Δ₀ ===
    print("\nComputing Δ₀ (scalar Laplacian)...")
    t0 = time.time()
    L0, eps0 = build_graph_laplacian(points, k=k)
    eigs_0 = compute_scalar_eigenvalues(L0, n_eigenvalues=10)
    time_0 = time.time() - t0
    
    # First nonzero eigenvalue
    lambda1_0_raw = eigs_0[1] if eigs_0[0] < 1e-10 else eigs_0[0]
    lambda1_0_scaled = lambda1_0_raw * scale_factor
    product_0 = lambda1_0_scaled * H_STAR
    
    print(f"  λ₁(Δ₀) raw = {lambda1_0_raw:.6f}")
    print(f"  λ₁(Δ₀) scaled = {lambda1_0_scaled:.6f}")
    print(f"  λ₁(Δ₀) × H* = {product_0:.4f}")
    print(f"  Time: {time_0:.2f}s")
    
    # === 1-Form Laplacian Δ₁ ===
    print("\nComputing Δ₁ (1-form Hodge Laplacian)...")
    t1 = time.time()
    L1, eps1, n_edges = build_hodge_laplacian_1form(points, k=k)
    eigs_1 = compute_1form_eigenvalues(L1, n_eigenvalues=20)
    time_1 = time.time() - t1
    
    # First nonzero eigenvalue (may have many zeros = harmonic 1-forms)
    # For K₇, b₁ = 0, so there should be few harmonic forms
    nonzero_mask = eigs_1 > 1e-8
    if nonzero_mask.any():
        lambda1_1_raw = eigs_1[nonzero_mask][0]
    else:
        lambda1_1_raw = eigs_1[1]
    
    lambda1_1_scaled = lambda1_1_raw * scale_factor
    product_1 = lambda1_1_scaled * H_STAR
    
    print(f"  n_edges = {n_edges}")
    print(f"  λ₁(Δ₁) raw = {lambda1_1_raw:.6f}")
    print(f"  λ₁(Δ₁) scaled = {lambda1_1_scaled:.6f}")
    print(f"  λ₁(Δ₁) × H* = {product_1:.4f}")
    print(f"  Time: {time_1:.2f}s")
    
    # Compute deviations
    dev_0_13 = abs(product_0 - TARGET_13) / TARGET_13 * 100
    dev_0_14 = abs(product_0 - TARGET_14) / TARGET_14 * 100
    dev_1_13 = abs(product_1 - TARGET_13) / TARGET_13 * 100
    dev_1_14 = abs(product_1 - TARGET_14) / TARGET_14 * 100
    
    return K7Result(
        N=N, k=k, ratio=ratio,
        lambda1_0_raw=lambda1_0_raw,
        lambda1_0_scaled=lambda1_0_scaled,
        product_0=product_0,
        deviation_0_from_13=dev_0_13,
        deviation_0_from_14=dev_0_14,
        lambda1_1_raw=lambda1_1_raw,
        lambda1_1_scaled=lambda1_1_scaled,
        product_1=product_1,
        deviation_1_from_13=dev_1_13,
        deviation_1_from_14=dev_1_14,
        n_edges=n_edges,
        time_0=time_0,
        time_1=time_1
    )

print("Experiment function defined.")

In [ ]:
# @title 9. Run Main Experiment

# Configuration
N_SAMPLES = 5000      # Increase for better accuracy (try 10000-20000 on A100)
K_NEIGHBORS = 30      # Number of nearest neighbors
RATIO = RATIO_CANONICAL  # = 33/28 ≈ 1.179

print("="*70)
print("K₇ LAPLACIAN COMPARISON: Δ₀ (SCALAR) vs Δ₁ (1-FORMS)")
print("="*70)
print(f"\nConfiguration:")
print(f"  N = {N_SAMPLES} samples")
print(f"  k = {K_NEIGHBORS} neighbors")
print(f"  ratio = {RATIO:.4f}")
print(f"  H* = {H_STAR}")
print(f"  Scale factor = {SCALE_FACTOR:.4f}")

# Run experiment
result = run_K7_experiment(
    N=N_SAMPLES,
    k=K_NEIGHBORS,
    ratio=RATIO,
    scale_factor=SCALE_FACTOR,
    seed=42
)

In [ ]:
# @title 10. Results Summary & Comparison

print("\n" + "="*70)
print("RESULTS SUMMARY")
print("="*70)

print(f"\n{'Metric':<25} {'Δ₀ (Scalar)':<20} {'Δ₁ (1-Forms)':<20}")
print("-"*70)
print(f"{'λ₁ (raw)':<25} {result.lambda1_0_raw:<20.6f} {result.lambda1_1_raw:<20.6f}")
print(f"{'λ₁ (scaled)':<25} {result.lambda1_0_scaled:<20.6f} {result.lambda1_1_scaled:<20.6f}")
print(f"{'λ₁ × H*':<25} {result.product_0:<20.4f} {result.product_1:<20.4f}")
print("-"*70)
print(f"{'Deviation from 13':<25} {result.deviation_0_from_13:<19.2f}% {result.deviation_1_from_13:<19.2f}%")
print(f"{'Deviation from 14':<25} {result.deviation_0_from_14:<19.2f}% {result.deviation_1_from_14:<19.2f}%")
print("-"*70)

# Determine winner
best_target_0 = TARGET_13 if result.deviation_0_from_13 < result.deviation_0_from_14 else TARGET_14
best_target_1 = TARGET_13 if result.deviation_1_from_13 < result.deviation_1_from_14 else TARGET_14
best_dev_0 = min(result.deviation_0_from_13, result.deviation_0_from_14)
best_dev_1 = min(result.deviation_1_from_13, result.deviation_1_from_14)

print(f"\n{'Best fit for Δ₀:':<25} {best_target_0} (deviation: {best_dev_0:.2f}%)")
print(f"{'Best fit for Δ₁:':<25} {best_target_1} (deviation: {best_dev_1:.2f}%)")

print("\n" + "="*70)
if best_dev_1 < best_dev_0:
    winner = "Δ₁ (1-FORMS)"
    improvement = best_dev_0 - best_dev_1
else:
    winner = "Δ₀ (SCALAR)"
    improvement = best_dev_1 - best_dev_0

print(f"WINNER: {winner}")
print(f"Improvement: {improvement:.2f} percentage points")
print("="*70)

# Physical interpretation
print("\n" + "-"*70)
print("PHYSICAL INTERPRETATION")
print("-"*70)
print(f"\nIf λ₁ × H* = {result.product_1:.2f} (from Δ₁):")
lambda1_physical = result.product_1 / H_STAR
mass_gap_MeV = lambda1_physical * 200  # Λ_QCD ≈ 200 MeV
print(f"  λ₁ = {lambda1_physical:.4f}")
print(f"  Δ_QCD = λ₁ × Λ_QCD = {lambda1_physical:.4f} × 200 MeV = {mass_gap_MeV:.1f} MeV")
print(f"\nGIFT prediction: Δ_QCD ≈ (14/99) × 200 = 28.3 MeV")

In [ ]:
# @title 11. Visualization

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot 1: Bar comparison
ax1 = axes[0]
x = ['Δ₀ (Scalar)', 'Δ₁ (1-Forms)']
products = [result.product_0, result.product_1]
colors = ['steelblue', 'darkorange']
bars = ax1.bar(x, products, color=colors, edgecolor='black', linewidth=1.5)
ax1.axhline(y=13, color='red', linestyle='--', linewidth=2, label='Target: 13')
ax1.axhline(y=14, color='green', linestyle='--', linewidth=2, label='Target: 14')
ax1.set_ylabel('λ₁ × H*', fontsize=12)
ax1.set_title('Laplacian Comparison on K₇', fontsize=14)
ax1.legend()
ax1.set_ylim([0, max(products) * 1.3])
for bar, val in zip(bars, products):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, 
             f'{val:.2f}', ha='center', fontsize=11, fontweight='bold')

# Plot 2: Deviation comparison
ax2 = axes[1]
x_dev = np.arange(2)
width = 0.35
dev_13 = [result.deviation_0_from_13, result.deviation_1_from_13]
dev_14 = [result.deviation_0_from_14, result.deviation_1_from_14]
bars1 = ax2.bar(x_dev - width/2, dev_13, width, label='From 13', color='red', alpha=0.7)
bars2 = ax2.bar(x_dev + width/2, dev_14, width, label='From 14', color='green', alpha=0.7)
ax2.set_ylabel('Deviation (%)', fontsize=12)
ax2.set_title('Deviation from Targets', fontsize=14)
ax2.set_xticks(x_dev)
ax2.set_xticklabels(['Δ₀', 'Δ₁'])
ax2.legend()

# Plot 3: GIFT Constants Context
ax3 = axes[2]
constants = ['dim(G₂)', 'dim(G₂)-1', 'b₂', 'b₃', 'H*']
values = [14, 13, 21, 77, 99]
colors_c = ['#2ecc71', '#e74c3c', '#3498db', '#9b59b6', '#f39c12']
bars3 = ax3.barh(constants, values, color=colors_c, edgecolor='black')
ax3.set_xlabel('Value', fontsize=12)
ax3.set_title('GIFT Topological Constants', fontsize=14)
for bar, val in zip(bars3, values):
    ax3.text(val + 1, bar.get_y() + bar.get_height()/2, 
             str(val), va='center', fontsize=11)

plt.tight_layout()
plt.savefig('K7_laplacian_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nFigure saved as 'K7_laplacian_comparison.png'")

In [ ]:
# @title 12. Ratio Sweep (Optional - finds optimal ratio)

RUN_RATIO_SWEEP = False  # Set to True to run (takes longer)

if RUN_RATIO_SWEEP:
    print("Running ratio sweep...")
    
    ratios = np.linspace(0.8, 1.6, 17)
    products_0 = []
    products_1 = []
    
    for r in ratios:
        res = run_K7_experiment(
            N=2000,  # Smaller N for speed
            k=20,
            ratio=r,
            scale_factor=SCALE_FACTOR,
            seed=42
        )
        products_0.append(res.product_0)
        products_1.append(res.product_1)
        print(f"ratio={r:.3f}: Δ₀→{res.product_0:.2f}, Δ₁→{res.product_1:.2f}")
    
    # Plot
    plt.figure(figsize=(10, 6))
    plt.plot(ratios, products_0, 'o-', label='Δ₀ (Scalar)', linewidth=2, markersize=8)
    plt.plot(ratios, products_1, 's-', label='Δ₁ (1-Forms)', linewidth=2, markersize=8)
    plt.axhline(y=13, color='red', linestyle='--', linewidth=2, label='Target: 13')
    plt.axhline(y=14, color='green', linestyle='--', linewidth=2, label='Target: 14')
    plt.axvline(x=RATIO_CANONICAL, color='purple', linestyle=':', linewidth=2, 
                label=f'Canonical: {RATIO_CANONICAL:.3f}')
    plt.xlabel('Ratio (H*/84)', fontsize=12)
    plt.ylabel('λ₁ × H*', fontsize=12)
    plt.title('λ₁ × H* vs TCS Ratio', fontsize=14)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('K7_ratio_sweep.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print("Ratio sweep skipped. Set RUN_RATIO_SWEEP = True to run.")

In [ ]:
# @title 13. Convergence Study (Optional - tests N dependence)

RUN_CONVERGENCE = False  # Set to True to run (takes longer)

if RUN_CONVERGENCE:
    print("Running convergence study...")
    
    N_values = [1000, 2000, 3000, 5000, 7500, 10000]
    products_0_conv = []
    products_1_conv = []
    
    for N in N_values:
        res = run_K7_experiment(
            N=N,
            k=30,
            ratio=RATIO_CANONICAL,
            scale_factor=SCALE_FACTOR,
            seed=42
        )
        products_0_conv.append(res.product_0)
        products_1_conv.append(res.product_1)
        print(f"N={N}: Δ₀→{res.product_0:.3f}, Δ₁→{res.product_1:.3f}")
    
    # Plot
    plt.figure(figsize=(10, 6))
    plt.plot(N_values, products_0_conv, 'o-', label='Δ₀ (Scalar)', linewidth=2, markersize=8)
    plt.plot(N_values, products_1_conv, 's-', label='Δ₁ (1-Forms)', linewidth=2, markersize=8)
    plt.axhline(y=13, color='red', linestyle='--', linewidth=2, label='Target: 13')
    plt.axhline(y=14, color='green', linestyle='--', linewidth=2, label='Target: 14')
    plt.xlabel('N (samples)', fontsize=12)
    plt.ylabel('λ₁ × H*', fontsize=12)
    plt.title('Convergence Study: λ₁ × H* vs Sample Size', fontsize=14)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('K7_convergence.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print("Convergence study skipped. Set RUN_CONVERGENCE = True to run.")

In [ ]:
# @title 14. Export Results

results_dict = {
    'config': {
        'N': result.N,
        'k': result.k,
        'ratio': result.ratio,
        'H_star': H_STAR,
        'scale_factor': SCALE_FACTOR,
    },
    'scalar_laplacian': {
        'lambda1_raw': result.lambda1_0_raw,
        'lambda1_scaled': result.lambda1_0_scaled,
        'product': result.product_0,
        'deviation_from_13': result.deviation_0_from_13,
        'deviation_from_14': result.deviation_0_from_14,
        'time_seconds': result.time_0,
    },
    '1form_laplacian': {
        'lambda1_raw': result.lambda1_1_raw,
        'lambda1_scaled': result.lambda1_1_scaled,
        'product': result.product_1,
        'deviation_from_13': result.deviation_1_from_13,
        'deviation_from_14': result.deviation_1_from_14,
        'n_edges': result.n_edges,
        'time_seconds': result.time_1,
    },
    'GIFT_constants': {
        'dim_G2': DIM_G2,
        'dim_K7': DIM_K7,
        'b2': B2,
        'b3': B3,
        'H_star': H_STAR,
        'det_g': DET_G,
    },
    'conclusion': {
        'winner': 'Delta_1' if min(result.deviation_1_from_13, result.deviation_1_from_14) < min(result.deviation_0_from_13, result.deviation_0_from_14) else 'Delta_0',
        'best_target_Delta0': 13 if result.deviation_0_from_13 < result.deviation_0_from_14 else 14,
        'best_target_Delta1': 13 if result.deviation_1_from_13 < result.deviation_1_from_14 else 14,
    }
}

# Save to JSON
with open('K7_laplacian_results.json', 'w') as f:
    json.dump(results_dict, f, indent=2)

print("Results saved to 'K7_laplacian_results.json'")
print("\n" + "="*70)
print("EXPERIMENT COMPLETE")
print("="*70)
print("\nFiles generated:")
print("  - K7_laplacian_comparison.png")
print("  - K7_laplacian_results.json")
if RUN_RATIO_SWEEP:
    print("  - K7_ratio_sweep.png")
if RUN_CONVERGENCE:
    print("  - K7_convergence.png")

In [ ]:
# @title 15. Quick Summary for Copy-Paste

summary = f"""
============================================================
K₇ LAPLACIAN COMPARISON - QUICK SUMMARY
============================================================

Configuration:
  N = {result.N} samples
  k = {result.k} neighbors
  ratio = {result.ratio:.4f}
  H* = {H_STAR}

Results:
  Δ₀ (Scalar):   λ₁ × H* = {result.product_0:.4f}
  Δ₁ (1-Forms):  λ₁ × H* = {result.product_1:.4f}

Deviations:
  Δ₀ from 13: {result.deviation_0_from_13:.2f}%  |  from 14: {result.deviation_0_from_14:.2f}%
  Δ₁ from 13: {result.deviation_1_from_13:.2f}%  |  from 14: {result.deviation_1_from_14:.2f}%

Winner: {'Δ₁ (1-Forms)' if min(result.deviation_1_from_13, result.deviation_1_from_14) < min(result.deviation_0_from_13, result.deviation_0_from_14) else 'Δ₀ (Scalar)'}

Physical prediction (from Δ₁):
  λ₁ = {result.product_1/H_STAR:.4f}
  Δ_QCD = {result.product_1/H_STAR * 200:.1f} MeV

GIFT target: Δ_QCD ≈ 28.3 MeV (λ₁ = 14/99)
============================================================
"""

print(summary)